### install spacy transformers

In [1]:
! pip install spacy[transformers]

### prepare training data
prepare NER training data using doccano tool
it should look like samples.json

In [2]:
from spacy.util import filter_spans
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

2023-05-19 16:26:16.683380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 16:26:17.313155: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19 16:26:17.313205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-19 16:26:17.313210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
with open('samples.json', 'r') as f:
    data = json.load(f)
    
print(data[0])

{'id': 6302, 'text': 'Yes, it is a product', 'label': [[13, 20, 'PRODUCT']], 'Comments': []}


In [7]:
training_data = {
    'classes' : ['PRODUCT', "SERVICE", "PROCESS"],
    'annotations' : []
}

for example in data:
  data_row = {}
  data_row['text'] = example['text']
  data_row['entities'] = []

  for annotation in example['label']:
    start = annotation[0]
    end = annotation[1]
    label = annotation[2]
    data_row['entities'].append((start, end, label))
  training_data['annotations'].append(data_row)
  
print(training_data['annotations'][1])

IndexError: list index out of range

In [ ]:
nlp = spacy.blank("en") 

doc_bin = DocBin()

In [ ]:
for training_row  in tqdm(training_data['annotations']): 
    text = training_row['text']
    labels = training_row['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

    
doc_bin.to_disk("training.spacy")

### run commands for training model

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

!python -m spacy train config.cfg --output ./ --paths.train ./training.spacy --paths.dev ./training.spacy

In [ ]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("It’s a complicated question, but let’s start with a product. It’s a complicated question, but let’s start with a service. It’s a complicated question, but let’s start with a process")

colors = {"PRODUCT": "#F67DE3", "MSERVICE": "#7DF6D9", "PROCESS":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)